
# Modelo de Classificação de escolas baseada em dados do INEP (2014-2023)

## Bibliotecas

In [2]:
import pandas as pd
import wbdata
import datetime
import pycountry
import geopandas


## Importação dos Dados

In [3]:
# Download latest version
path = "./dataset/master.csv"

# Carregar o arquivo CSV no DataFrame
df = pd.read_csv(path)


## Análise Exploratória de Dados

In [4]:
df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [5]:
# Remover coluna country-year por motivos de redundância
df = df.drop(columns=["country-year"])

In [6]:
antes = len(df)

# Removendo Duplicatas
df.drop_duplicates(ignore_index=True,inplace=True)

depois = len(df)

print(f"Linhas duplicadas existentes e removidas = {depois - antes}")

Linhas duplicadas existentes e removidas = 0


- Informações sobre Dataframe

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country             27820 non-null  object 
 1   year                27820 non-null  int64  
 2   sex                 27820 non-null  object 
 3   age                 27820 non-null  object 
 4   suicides_no         27820 non-null  int64  
 5   population          27820 non-null  int64  
 6   suicides/100k pop   27820 non-null  float64
 7   HDI for year        8364 non-null   float64
 8    gdp_for_year ($)   27820 non-null  object 
 9   gdp_per_capita ($)  27820 non-null  int64  
 10  generation          27820 non-null  object 
dtypes: float64(2), int64(4), object(5)
memory usage: 2.3+ MB


- Estatística Básica

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
year,27820.0,2.001258e+03,8.469055e+00,1985.000,1995.000,2002.000,2008.000,2.016000e+03
suicides_no,27820.0,2.425744e+02,9.020479e+02,0.000,3.000,25.000,131.000,2.233800e+04
population,27820.0,1.844794e+06,3.911779e+06,278.000,97498.500,430150.000,1486143.250,4.380521e+07
suicides/100k pop,27820.0,1.281610e+01,1.896151e+01,0.000,0.920,5.990,16.620,2.249700e+02
HDI for year,8364.0,7.766011e-01,9.336671e-02,0.483,0.713,0.779,0.855,9.440000e-01
gdp_per_capita ($),27820.0,1.686646e+04,1.888758e+04,251.000,3447.000,9372.000,24874.000,1.263520e+05


- Contando Nulos ou NaN

In [9]:
len(df)

27820

In [10]:
df.isnull().sum()

country                   0
year                      0
sex                       0
age                       0
suicides_no               0
population                0
suicides/100k pop         0
HDI for year          19456
 gdp_for_year ($)         0
gdp_per_capita ($)        0
generation                0
dtype: int64

In [11]:
df.head(20)

,country,year,sex,age,suicides_no,population,suicides/100k pop,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,NaN,"2,156,624,900",796,Boomers
5,Albania,1987,female,75+ years,1,35600,2.81,NaN,"2,156,624,900",796,G.I. Generation
6,Albania,1987,female,35-54 years,6,278800,2.15,NaN,"2,156,624,900",796,Silent
7,Albania,1987,female,25-34 years,4,257200,1.56,NaN,"2,156,624,900",796,Boomers
8,Albania,1987,male,55-74 years,1,137500,0.73,NaN,"2,156,624,900",796,G.I. Generation
9,Albania,1987,female,5-14 years,0,311000,0.00,NaN,"2,156,624,900",796,Generation X


- Preencher os dados nulos com informações do World Bank

In [12]:
# Função para mapear o nome do país para seu código ISO (2 letras)
def get_country_code(country_name):
    try:
        country = pycountry.countries.get(name=country_name)
        return country.alpha_2  # Código ISO de 2 letras
    except AttributeError:
        return None  # Retorna None se o país não for encontrado

# Função para buscar PIB
def get_gdp_data(country_code, year):
    indicator = {'NY.GDP.MKTP.CD': 'GDP'}
    start_date = datetime.datetime(year, 1, 1)
    end_date = datetime.datetime(year, 12, 31)
    
    try:
        gdp_data = wbdata.get_dataframe(indicator, country=country_code, data_date=(start_date, end_date))
        return gdp_data['GDP'][0]
    except:
        return None

# Exemplo: Preenchendo com PIB de uma API
for index, row in df[df['HDI for year'].isnull()].iterrows():
    country = row['country']
    year = row['year']
    country_code = get_country_code(country)  # Aqui você precisaria de uma função para buscar o código ISO
    gdp_value = get_gdp_data(country_code, year)
    
    if gdp_value:
        df.at[index, 'HDI for year'] = gdp_value  # Preenche o valor nulo com o PIB


In [13]:
hdr = pd.read_csv('./dataset/HDR.csv')
hdr.head()

,iso3,country,hdicode,region,hdi_rank_2022,hdi_1990,hdi_1991,hdi_1992,hdi_1993,hdi_1994,...,pop_total_2013,pop_total_2014,pop_total_2015,pop_total_2016,pop_total_2017,pop_total_2018,pop_total_2019,pop_total_2020,pop_total_2021,pop_total_2022
0,AFG,Afghanistan,Low,SA,182.0,0.284,0.292,0.299,0.307,0.300,...,31.541208,32.716210,33.753499,34.636207,35.643417,36.686784,37.769498,38.972231,40.099462,41.128771
1,ALB,Albania,High,ECA,74.0,0.649,0.632,0.616,0.618,0.623,...,2.887014,2.884102,2.882480,2.881063,2.879355,2.877013,2.873883,2.866849,2.854710,2.842321
2,DZA,Algeria,High,AS,93.0,0.593,0.596,0.601,0.602,0.603,...,38.000627,38.760168,39.543154,40.339329,41.136546,41.927007,42.705368,43.451666,44.177968,44.903225
3,AND,Andorra,Very High,NaN,35.0,NaN,NaN,NaN,NaN,NaN,...,0.071366,0.071622,0.071746,0.072540,0.073836,0.075013,0.076343,0.077700,0.079034,0.079824
4,AGO,Angola,Medium,SSA,150.0,NaN,NaN,NaN,NaN,NaN,...,26.147002,27.128336,28.127721,29.154746,30.208628,31.273533,32.353588,33.428486,34.503774,35.588987


In [46]:
hdr_dropped = hdr.drop(columns=['hdicode','region','hdi_rank_2022','iso3'])
hdr_final = hdr_dropped.drop(hdr_dropped.columns[34:],axis=1)
hdr_final.columns = hdr_final.columns.str.replace('hdi_','',regex=False)
hdr_final.reset_index()
hdr_final.head()

,country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,0.284,0.292,0.299,0.307,0.300,0.318,0.326,0.330,0.329,...,0.475,0.480,0.479,0.483,0.485,0.486,0.492,0.488,0.473,0.462
1,Albania,0.649,0.632,0.616,0.618,0.623,0.632,0.644,0.641,0.656,...,0.793,0.797,0.797,0.795,0.796,0.797,0.800,0.784,0.785,0.789
2,Algeria,0.593,0.596,0.601,0.602,0.603,0.607,0.618,0.626,0.635,...,0.729,0.732,0.736,0.738,0.738,0.740,0.742,0.730,0.740,0.745
3,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.851,0.854,0.856,0.859,0.860,0.863,0.865,0.843,0.855,0.884
4,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.555,0.565,0.591,0.595,0.597,0.598,0.597,0.594,0.590,0.591


In [47]:
hdr_final_melted = hdr_final.melt(id_vars='country',var_name='year',value_name='HDI for year')
hdr_final_melted['year'] = hdr_final_melted['year'].astype(int)

In [50]:
merged_df = df.merge(hdr_final_melted,on=['country', 'year'],how='left')

print(merged_df.columns)

df['HDI for year'] = merged_df['HDI for year_x'].combine_first(merged_df['HDI for year_y'])

Index(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides/100k pop', 'HDI for year_x', ' gdp_for_year ($) ',
       'gdp_per_capita ($)', 'generation', 'HDI for year_y'],
      dtype='object')


In [51]:
df

,country,year,sex,age,suicides_no,population,suicides/100k pop,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,NaN,"2,156,624,900",796,Boomers
...,...,...,...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54 years,107,3620833,2.96,0.675,"63,067,077,179",2309,Generation X
27816,Uzbekistan,2014,female,75+ years,9,348465,2.58,0.675,"63,067,077,179",2309,Silent
27817,Uzbekistan,2014,male,5-14 years,60,2762158,2.17,0.675,"63,067,077,179",2309,Generation Z
27818,Uzbekistan,2014,female,5-14 years,44,2631600,1.67,0.675,"63,067,077,179",2309,Generation Z


In [52]:
df.isnull().sum()

country                  0
year                     0
sex                      0
age                      0
suicides_no              0
population               0
suicides/100k pop        0
HDI for year          4954
 gdp_for_year ($)        0
gdp_per_capita ($)       0
generation               0
dtype: int64